In [1]:
%matplotlib inline

# 加载需要的库
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.datasets import StuperDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import  load_preproc_data_stuper

from aif360.algorithms.inprocessing.adversarial_debiasing import FairEduNet

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html


#### 读取数据集和设置选项

In [2]:
# 获取数据集，进行训练集和测试集的划分
dataset_orig = load_preproc_data_stuper()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.9], shuffle=False)

In [3]:
# 打印出数据集的一些特征
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(584, 41)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['sex', 'age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'schoolsup_encoded', 'famsup_encoded', 'paid_encoded', 'activities_encoded', 'nursery_encoded', 'higher_encoded', 'internet_encoded', 'romantic_encoded', 'school_MS', 'address_U', 'famsize_LE3', 'Pstatus_T', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother', 'guardian_other', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher']


#### 原始训练数据的指标

In [4]:
# 原始数据集的指标
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.151784
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.052296


In [5]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
# 缩放数据集 - 验证缩放是否不会影响组标签统计数据
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.151784
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.052296


### 学习计划分类器，无需去偏见

In [6]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [7]:
plain_model.fit(dataset_orig_train)

epoch 0; iter: 3; batch classifier mean loss: 0.661435
epoch 1; iter: 3; batch classifier mean loss: 0.533345
epoch 2; iter: 3; batch classifier mean loss: 0.453167
epoch 3; iter: 3; batch classifier mean loss: 0.407376
epoch 4; iter: 3; batch classifier mean loss: 0.359507
epoch 5; iter: 3; batch classifier mean loss: 0.309493
epoch 6; iter: 3; batch classifier mean loss: 0.284718
epoch 7; iter: 3; batch classifier mean loss: 0.292950
epoch 8; iter: 3; batch classifier mean loss: 0.259302
epoch 9; iter: 3; batch classifier mean loss: 0.206016
epoch 10; iter: 3; batch classifier mean loss: 0.162740
epoch 11; iter: 3; batch classifier mean loss: 0.144100
epoch 12; iter: 3; batch classifier mean loss: 0.128121
epoch 13; iter: 3; batch classifier mean loss: 0.100032
epoch 14; iter: 3; batch classifier mean loss: 0.157014
epoch 15; iter: 3; batch classifier mean loss: 0.177179
epoch 16; iter: 3; batch classifier mean loss: 0.073843
epoch 17; iter: 3; batch classifier mean loss: 0.087650
ep

In [8]:
# 将朴素模型应用于测试数据
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [9]:
# 朴素模型数据集的度量指标（未去偏见）
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.151784
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.177296


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.876923
Test set: Balanced classification accuracy = 0.875237
Test set: Disparate impact = 0.638021
Test set: Equal opportunity difference = 0.000000
Test set: Average odds difference = -0.140357
Test set: Theil_index = 0.097267


### 应用基于Adversarial Debiasing的处理中算法

In [10]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [11]:
# Learn parameters with debias set to True
# 在去偏见设置为True时学习参数
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess,
                          num_epochs=100)

In [12]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 3; batch classifier mean loss: 0.638059; batch adversarial mean loss: 0.691876
epoch 1; iter: 3; batch classifier mean loss: 0.503656; batch adversarial mean loss: 0.685790
epoch 2; iter: 3; batch classifier mean loss: 0.456047; batch adversarial mean loss: 0.687217
epoch 3; iter: 3; batch classifier mean loss: 0.416042; batch adversarial mean loss: 0.697506
epoch 4; iter: 3; batch classifier mean loss: 0.353159; batch adversarial mean loss: 0.686744
epoch 5; iter: 3; batch classifier mean loss: 0.310251; batch adversarial mean loss: 0.684307
epoch 6; iter: 3; batch classifier mean loss: 0.413262; batch adversarial mean loss: 0.681708
epoch 7; iter: 3; batch classifier mean loss: 0.394231; batch adversarial mean loss: 0.683374
epoch 8; iter: 3; batch classifier mean loss: 0.541277; batch adversarial mean loss: 0.642848
epoch 9; iter: 3; batch classifier mean loss: 0.459646; batch adversarial mean loss: 0.697707
epoch 10; iter: 3; batch classifier mean loss: 0.357230; bat

epoch 88; iter: 3; batch classifier mean loss: 0.142778; batch adversarial mean loss: 0.684917
epoch 89; iter: 3; batch classifier mean loss: 0.145354; batch adversarial mean loss: 0.683135
epoch 90; iter: 3; batch classifier mean loss: 0.135888; batch adversarial mean loss: 0.685983
epoch 91; iter: 3; batch classifier mean loss: 0.127344; batch adversarial mean loss: 0.687124
epoch 92; iter: 3; batch classifier mean loss: 0.120164; batch adversarial mean loss: 0.686232
epoch 93; iter: 3; batch classifier mean loss: 0.106697; batch adversarial mean loss: 0.685535
epoch 94; iter: 3; batch classifier mean loss: 0.143915; batch adversarial mean loss: 0.681896
epoch 95; iter: 3; batch classifier mean loss: 0.117295; batch adversarial mean loss: 0.682944
epoch 96; iter: 3; batch classifier mean loss: 0.086398; batch adversarial mean loss: 0.680587
epoch 97; iter: 3; batch classifier mean loss: 0.091212; batch adversarial mean loss: 0.683343
epoch 98; iter: 3; batch classifier mean loss: 0.1

In [13]:
# 将朴素模型应用于测试数据
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [14]:
# 朴素模型数据集的度量指标（不去偏见）
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# # 去偏见的模型数据集的度量指标
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)

# print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
# print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
# print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
# print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
# print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())
print("Test set: Statistical parity difference = %f" % classified_metric_nodebiasing_test.statistical_parity_difference())
print("Test set: Equalized Odds difference = %f" % classified_metric_nodebiasing_test.equalized_odds_difference())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())


display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)

print("Test set: Statistical parity difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())
print("Test set: Equalized Odds difference = %f" % classified_metric_debiasing_test.equalized_odds_difference())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.151784
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.177296


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.035018
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.014031


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.876923
Test set: Statistical parity difference = -0.177296
Test set: Equalized Odds difference = -0.160714
Test set: Equal opportunity difference = 0.000000
Test set: Disparate impact = 0.638021


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.846154
Test set: Statistical parity difference = -0.014031
Test set: Equalized Odds difference = 0.047619
Test set: Equal opportunity difference = 0.000000
Test set: Disparate impact = 0.957031
